<a href="https://colab.research.google.com/github/sai-phyo-hein/Hybrid_Transformer_for_Portfolio_Allocation/blob/main/Bench_Vanilla_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import duckdb as ddb
import pandas as pd
import datetime

import numpy as np
import math

from sklearn.preprocessing import StandardScaler
import tensorflow as tf

In [2]:
date_series = ['2014-01-01']
while datetime.date.fromisoformat(date_series[-1]) < datetime.date.fromisoformat('2023-03-28'):
    date_series.append(
        (datetime.date.fromisoformat(date_series[-1]) + datetime.timedelta(days = 1)).strftime('%Y-%m-%d')
    )

In [3]:
selected_eq = [
    'AXISBANK', 'BHARTIARTL', 'HDFCBANK', 'ICICIBANK', 'INFY', 'ITC', 'LT',
    'RELIANCE', 'SBIN', 'TATAMOTORS', 'TATASTEEL', 'TCS'
]
con = ddb.connect('/content/drive/MyDrive/MScFE Capstone Project /MScFE_Capstone_Project_Group_Work/DataSource/nifty_50_dataset.db')
data = pd.DataFrame()
data['timestamp'] = date_series
for i, ticker in enumerate(selected_eq):
    temp = con.sql(f'select CH_TIMESTAMP as timestamp, CH_CLOSING_PRICE as {ticker} from {ticker}').df().drop_duplicates('timestamp')
    data = pd.merge(data, temp, how = 'outer', on = 'timestamp')
    print(ticker, data.shape)
con.close()

AXISBANK (3622, 2)
BHARTIARTL (3622, 3)
HDFCBANK (3622, 4)
ICICIBANK (3622, 5)
INFY (3622, 6)
ITC (3622, 7)
LT (3622, 8)
RELIANCE (3622, 9)
SBIN (3622, 10)
TATAMOTORS (3622, 11)
TATASTEEL (3622, 12)
TCS (3622, 13)


In [4]:
data['timestamp'] = pd.to_datetime(data.timestamp, format = '%Y-%m-%d')
data.set_index('timestamp', inplace = True)
data.dropna(how = 'all', inplace = True)
data.isnull().sum()

AXISBANK      0
BHARTIARTL    0
HDFCBANK      0
ICICIBANK     0
INFY          0
ITC           0
LT            0
RELIANCE      0
SBIN          0
TATAMOTORS    0
TATASTEEL     0
TCS           0
dtype: int64

In [5]:
data = data.astype(float)

In [6]:
data.head()

,AXISBANK,BHARTIARTL,HDFCBANK,ICICIBANK,INFY,ITC,LT,RELIANCE,SBIN,TATAMOTORS,TATASTEEL,TCS
timestamp,,,,,,,,,,,,
2014-01-01,1292.20,337.75,665.15,1097.7,3468.00,322.40,1068.55,888.80,1765.05,374.95,425.9,2153.30
2014-01-02,1272.30,328.70,657.00,1075.6,3480.55,314.95,1036.05,875.00,1742.60,372.35,419.2,2167.00
2014-01-03,1260.90,329.35,663.10,1066.8,3565.15,314.75,1012.05,864.25,1719.65,362.70,412.5,2222.20
2014-01-06,1256.30,329.35,661.70,1040.8,3517.90,315.65,1010.50,854.95,1688.15,366.95,408.3,2239.60
2014-01-07,1234.25,331.70,664.40,1050.0,3457.15,316.70,1019.50,842.10,1661.40,365.10,394.7,2206.15


### Downloading Data

In [7]:
date_series = ['2014-01-01']
while datetime.date.fromisoformat(date_series[-1]) < datetime.date.fromisoformat('2023-03-28'):
    date_series.append(
        (datetime.date.fromisoformat(date_series[-1]) + datetime.timedelta(days = 1)).strftime('%Y-%m-%d')
    )

In [8]:
selected_eq = [
    'AXISBANK', 'BHARTIARTL', 'HDFCBANK', 'ICICIBANK', 'INFY', 'ITC', 'LT',
    'RELIANCE', 'SBIN', 'TATAMOTORS', 'TATASTEEL', 'TCS'
]
con = ddb.connect('/content/drive/MyDrive/MScFE Capstone Project /MScFE_Capstone_Project_Group_Work/DataSource/nifty_50_dataset.db')
data = pd.DataFrame()
data['timestamp'] = date_series
for i, ticker in enumerate(selected_eq):
    temp = con.sql(f'select CH_TIMESTAMP as timestamp, CH_CLOSING_PRICE as {ticker} from {ticker}').df().drop_duplicates('timestamp')
    data = pd.merge(data, temp, how = 'outer', on = 'timestamp')
    print(ticker, data.shape)
con.close()

AXISBANK (3622, 2)
BHARTIARTL (3622, 3)
HDFCBANK (3622, 4)
ICICIBANK (3622, 5)
INFY (3622, 6)
ITC (3622, 7)
LT (3622, 8)
RELIANCE (3622, 9)
SBIN (3622, 10)
TATAMOTORS (3622, 11)
TATASTEEL (3622, 12)
TCS (3622, 13)


In [9]:
data['timestamp'] = pd.to_datetime(data.timestamp, format = '%Y-%m-%d')
data.set_index('timestamp', inplace = True)
data.dropna(how = 'all', inplace = True)
data.isnull().sum()

AXISBANK      0
BHARTIARTL    0
HDFCBANK      0
ICICIBANK     0
INFY          0
ITC           0
LT            0
RELIANCE      0
SBIN          0
TATAMOTORS    0
TATASTEEL     0
TCS           0
dtype: int64

In [10]:
data = data.astype(float)

In [11]:
data.head()

,AXISBANK,BHARTIARTL,HDFCBANK,ICICIBANK,INFY,ITC,LT,RELIANCE,SBIN,TATAMOTORS,TATASTEEL,TCS
timestamp,,,,,,,,,,,,
2014-01-01,1292.20,337.75,665.15,1097.7,3468.00,322.40,1068.55,888.80,1765.05,374.95,425.9,2153.30
2014-01-02,1272.30,328.70,657.00,1075.6,3480.55,314.95,1036.05,875.00,1742.60,372.35,419.2,2167.00
2014-01-03,1260.90,329.35,663.10,1066.8,3565.15,314.75,1012.05,864.25,1719.65,362.70,412.5,2222.20
2014-01-06,1256.30,329.35,661.70,1040.8,3517.90,315.65,1010.50,854.95,1688.15,366.95,408.3,2239.60
2014-01-07,1234.25,331.70,664.40,1050.0,3457.15,316.70,1019.50,842.10,1661.40,365.10,394.7,2206.15


In [12]:
data.columns

Index(['AXISBANK', 'BHARTIARTL', 'HDFCBANK', 'ICICIBANK', 'INFY', 'ITC', 'LT',
       'RELIANCE', 'SBIN', 'TATAMOTORS', 'TATASTEEL', 'TCS'],
      dtype='object')

### Data Preparation for Model

In [13]:
#Return calculation
def ReturnCalculation (Database,lag):
    dimension=Database.shape[0];dif=lag;Out=np.zeros([dimension-dif])
    for i in range(dimension-dif):
        Out[i]=(np.log(Database[i+dif])-np.log(Database[i]))
    return np.append(np.repeat(np.nan, dif),Out)

#STD Calculation
def SDCalculation (DailyReturns, LagSD):
    dimension=DailyReturns.shape[0]; dif=LagSD; Out=np.zeros([dimension-dif])
    for i in range (dimension-dif):
        Out[i]=np.std(DailyReturns[i:i+LagSD],ddof=1)
    return np.append(np.repeat(np.nan, dif),Out)

In [14]:
return_data = pd.DataFrame(index = data.index)
for col in data.columns:
    return_data[col + '_lag_return']  = ReturnCalculation(data[col], 5)
    return_data[col + '_lag_sd'] = SDCalculation(data[col], 5)
    return_data[col + '_true_return'] = ReturnCalculation(data[col], 1)
return_data.dropna(inplace  =True)

In [15]:
xdata = return_data[[col for col in return_data.columns if 'true' not in col]]
ydata = return_data[[col for col in return_data.columns if 'true' in col]]
xdata.shape, ydata.shape

((2527, 24), (2527, 12))

In [16]:
scaler = StandardScaler().fit(xdata)
scaled_xdata = scaler.transform(xdata)
scaled_xdata

array([[-0.72205919,  0.25809263, -0.37485714, ...,  0.04524391,
         0.73936225, -0.00368889],
       [-0.80081029,  0.34340558,  0.02298925, ...,  0.03123295,
         0.6938356 , -0.17133434],
       [-0.89979993,  0.5759377 ,  0.04219102, ..., -0.05100363,
         0.52863644, -0.48332398],
       ...,
       [ 0.27306036, -0.18892427, -0.29785716, ..., -0.55232703,
        -0.8355729 ,  0.2377543 ],
       [ 0.24239934, -0.14658374, -0.00209913, ..., -0.54457596,
        -0.91308532,  0.47889926],
       [ 0.00649204, -0.26166517, -0.08655103, ..., -0.5421701 ,
        -0.19824492,  0.0313815 ]])

In [17]:
def position_encoding_init(n_position, d_pos_vec):
    position_enc = np.array([
        [math.pi*(pos/(n_position-1)) for i in range(d_pos_vec)]
        if pos != 0 else np.zeros(d_pos_vec) for pos in range(n_position)])
    return np.cos(position_enc)

def transformer_database(timestep, xdata, ydata):
    features = xdata.shape[1]
    sample = xdata.shape[0] - timestep + 1
    output = ydata.shape[1]
    XDataTrainScaledRNN = np.zeros([sample, timestep, features])
    pos_encoding=position_encoding_init(timestep, xdata.shape[1])
    YDataTrainRNN = ydata.iloc[timestep - 1:, :].values
    for i in range(sample):
        XDataTrainScaledRNN[i, :, :] = xdata[i:(timestep + i)] + pos_encoding
    return tf.convert_to_tensor(XDataTrainScaledRNN, np.float32), tf.convert_to_tensor(YDataTrainRNN, np.float32)

In [18]:
timestep = 16

In [19]:
xtrainRNN, ytrainRNN = transformer_database(timestep, scaled_xdata, ydata)

In [20]:
xtrainRNN[0, :, :][0]

<tf.Tensor: shape=(24,), dtype=float32, numpy=
array([ 0.2779408 ,  1.2580926 ,  0.6251429 ,  0.60820764,  0.94107896,
        0.43127656,  0.5280039 ,  1.4206057 ,  0.85715216,  1.4006661 ,
        0.5581461 ,  0.80579853, -0.38371873,  0.97127616,  0.09559315,
        0.76615757,  0.460198  ,  1.5710223 ,  0.8268885 ,  0.77711815,
        0.2907853 ,  1.0452439 ,  1.7393622 ,  0.9963111 ], dtype=float32)>

### Model Architecture

The architecture of the model is structured as follows by inspiring the work of  Eduardo Ramos-Pérez, Pablo J. Alonso-González and José Javier Núñez-Velázquez.

In [21]:
###MultiHeadSelfAttention
class MultiHeadSelfAttention(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads = 8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embedding dimensions = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
        self.projection_dim = embed_dim // num_heads
        self.query_dense = tf.keras.layers.Dense(embed_dim)
        self.key_dense = tf.keras.layers.Dense(embed_dim)
        self.value_dense = tf.keras.layers.Dense(embed_dim)
        self.combine_heads = tf.keras.layers.Dense(embed_dim)
    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b = True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis = -1)
        output = tf.matmul(weights, value)
        return output, weights
    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm = [0, 2, 1, 3])
    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)  # (batch_size, seq_len, embed_dim)
        key = self.key_dense(inputs)  # (batch_size, seq_len, embed_dim)
        value = self.value_dense(inputs)  # (batch_size, seq_len, embed_dim)
        query = self.separate_heads(query, batch_size)  # (batch_size, num_heads, seq_len, projection_dim)
        key = self.separate_heads(key, batch_size)  # (batch_size, num_heads, seq_len, projection_dim)
        value = self.separate_heads(value, batch_size)  # (batch_size, num_heads, seq_len, projection_dim)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len, num_heads, projection_dim)
        concat_attention = tf.reshape(attention, (batch_size, -1, self.embed_dim))  # (batch_size, seq_len, embed_dim)
        output = self.combine_heads(concat_attention)  # (batch_size, seq_len, embed_dim)
        return output


In the HybridTransformer_Block, the input is firstly processed with the LSTM layer in replace of positional encoding in Vanilla Transformer. The sharpe loss function is implemented as an objective function for the model. Before the output is optimized to update the layer parameters, the output is processed for weight constraint where the sum of all weights in a portfolio must be equal to 1. The output is then clipped so that the users can enhance the distribution of weight without puting all investment in a single asset whenever that asset has higher average return and higher risk.

In [31]:
###Transformer Keras Block
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate = 0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = tf.keras.Sequential([tf.keras.layers.Dense(ff_dim, activation = "relu"), tf.keras.layers.Dense(embed_dim),])
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon = 1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon = 1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training = training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training = training)
        return self.layernorm2(out1 + ffn_output)


class HybridTransformer_Portfolio(tf.keras.layers.Layer):
    def __init__(self, shape1, shape2, outputShape, headsAttention, dropout, learningRate, priceData, ub, lb):
        self.shape1 = shape1
        self.shape2 = shape2
        self.outputShape = outputShape
        self.headsAttention = headsAttention
        self.dropout = dropout
        self.learningRate = learningRate
        self.priceData = priceData
        self.model = None
        self.ub = ub
        self.lb = lb

    def rebalance(self, weight, lb, ub):
        old = weight
        weight_clamped = tf.clip_by_value(old, lb, ub)
        while True:
            leftover = (old - weight_clamped).sum().item()
            nominees = weight_clamped[tf.where(weight_clamped != ub)[0]]
            gift = leftover * (nominees / nominees.sum())
            weight_clamped[tf.where(weight_clamped != ub)[0]] += gift
            old = weight_clamped
            if len(tf.where(weight_clamped > ub)[0]) ==0:
                break
            else:
                weight_clamped = tf.clip_by_value(old, lb, ub)


    def Transformer_Model(self):
        #Model Structure is defined
        Input = tf.keras.Input(shape = (self.shape1, self.shape2), name = 'Input')
        #LSTM is applied on top of the transformer
        #Pos_Encoding is Used for Vanilla Transformer
        #X = tf.keras.layers.LSTM(units = 16, dropout = Dropout, return_sequences = True)(Input)
        #Transformer architecture is implemented
        transformer_block_1 = TransformerBlock(embed_dim = 24, num_heads=self.headsAttention, ff_dim = 8, rate = self.dropout, )
        X = transformer_block_1(Input)

        #Dense layers are used
        X = tf.keras.layers.GlobalAveragePooling1D()(X)
        X = tf.keras.layers.Dense(8, activation=tf.nn.sigmoid)(X)
        X = tf.keras.layers.Dropout(Dropout)(X)
        Output = tf.keras.layers.Dense(self.outputShape, activation=tf.nn.softmax, name="Output")(X)

        # scaling for the constraints sum = 1
        Output = tf.math.divide(Output, tf.reduce_sum(Output, axis = 1, keepdims=True))

        # clip the output for addressing weight bounds
        Output = tf.clip_by_value(Output, clip_value_min = self.lb, clip_value_max = self.ub)

        model = tf.keras.Model(inputs=Input, outputs=Output)
        #Optimizer is defined
        Opt = tf.keras.optimizers.Adam(learning_rate=LearningRate, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name='Adam')

        #Configuring Custom Loss Funciton with Mean Sharpe Ratio
        def sharpe_loss(_, y_pred):
            data = tf.divide(self.priceData, self.priceData[0])
            y_pred = tf.unstack(y_pred)
            sharpes = tf.zeros((1,1))
            for y in y_pred:
                portfolio_values = tf.reduce_sum(tf.multiply(data, y), axis=1, )
                portfolio_returns = (portfolio_values[1:] - portfolio_values[:-1]) / portfolio_values[:-1]  # % change formula
                sharpe = tf.keras.backend.mean(portfolio_returns) / tf.keras.backend.std(portfolio_returns)
                sharpes = tf.concat((sharpes, tf.reshape(sharpe, (1, -1))), axis = 0)
            return -tf.keras.backend.mean(sharpes[0][1:])

        #Model is compiled
        model.compile(optimizer=Opt, loss= sharpe_loss)
        return model

    def allocation(self, xtrainRNN, ytrainRNN, Epochs, BatchSize):
        if self.model == None:
            self.model = self.Transformer_Model()
            self.model.fit(xtrainRNN, ytrainRNN, epochs = Epochs, verbose = 0, batch_size = BatchSize)
            return self.model.predict(xtrainRNN)

In [32]:
Dropout = 0.05
LearningRate = 0.01
Epochs = 1000
Alpha = 0.005
DF = 4
BatchSize = 16
lb = 0
ub = 0.3
priceData = tf.convert_to_tensor(data.loc[xdata.iloc[timestep - 1 : timestep, :].index[0]:, :].values, np.float32)
model = HybridTransformer_Portfolio(xtrainRNN.shape[1], xtrainRNN.shape[2], outputShape = ytrainRNN.shape[1],  headsAttention = 4, dropout= Dropout, learningRate = LearningRate, priceData = priceData, lb= 0.01, ub = 0.2)
weights = model.allocation(xtrainRNN, ytrainRNN, Epochs = Epochs, BatchSize = BatchSize)

79/79 [==============================] - 0s 2ms/step


In [33]:
weights

array([[0.1104794 , 0.06375045, 0.05615333, ..., 0.04395669, 0.16761413,
        0.10953581],
       [0.11185599, 0.06469423, 0.05767881, ..., 0.04397224, 0.16857462,
        0.10573842],
       [0.11462917, 0.06472468, 0.05854079, ..., 0.04397188, 0.16888022,
        0.10350251],
       ...,
       [0.09169663, 0.07897653, 0.06791159, ..., 0.03507238, 0.15948759,
        0.09791385],
       [0.08989351, 0.08440382, 0.06741456, ..., 0.03484925, 0.16105258,
        0.09176052],
       [0.09026007, 0.08503878, 0.06579529, ..., 0.03492308, 0.16303582,
        0.09062966]], dtype=float32)

In [35]:
weights[-1] * 100

array([ 9.026008 ,  8.503878 ,  6.579529 ,  7.169561 , 12.221941 ,
        5.5880775,  8.230191 ,  7.4002924,  6.42166  ,  3.4923081,
       16.303583 ,  9.062966 ], dtype=float32)

After running the training and then repredict the xtrain data (which will be replaced on backtesting and comparison step), the model suggests the weight allocation as in above for the optimum sharpe ratio in each step.

In [36]:
weights.shape

(2512, 12)

In [37]:
ytrainRNN.shape

TensorShape([2512, 12])